## Overview of MLB Homeruns Dataset

In [ ]:
import pandas as pd
mlb_homeruns_df = pd.read_csv('/kaggle/input/mlb-fan-content-interaction/2016-mlb-homeruns.csv')

In [ ]:
mlb_homeruns_df.head()

# Data Preprocessing & Training in Batches

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os
import requests
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout, LSTM, TimeDistributed
from tensorflow.keras.optimizers import Adam
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import tensorflow.keras.backend as K
import gc

# Load pre-trained CNN model (feature extractor)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze pre-trained layers
feature_extractor = Model(inputs=base_model.input, outputs=Flatten()(base_model.output))  # Extracted features

# Define the LSTM-based regression model
model = Sequential([
    TimeDistributed(feature_extractor, input_shape=(50, 224, 224, 3)),  # Apply CNN to each frame
    LSTM(256, return_sequences=False),  # LSTM processes extracted features
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(3)  # Predict ExitVelocity, HitDistance, LaunchAngle
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# Function to download video
def download_video(video_url, save_path):
    response = requests.get(video_url, stream=True)
    with open(save_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=1024):
            file.write(chunk)
    return save_path

# Function to extract frames from video
def preprocess_video(video_url, save_path, frame_count=50):
    video_path = download_video(video_url, f'{save_path}.mp4')
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_ids = np.linspace(0, total_frames - 1, frame_count, dtype=int)  # Sample evenly
    
    for fid in frame_ids:
        cap.set(cv2.CAP_PROP_POS_FRAMES, fid)
        ret, frame = cap.read()
        if not ret:
            continue
        frame = cv2.resize(frame, (224, 224)) / 255.0  # Normalize
        frames.append(frame)
    
    cap.release()
    os.remove(video_path)  # Remove temp file
    return np.array(frames) if len(frames) == frame_count else None  # Ensure 10 frames

# Load and merge datasets
df1 = pd.read_csv('/kaggle/input/mlb-fan-content-interaction/2016-mlb-homeruns.csv')
df2 = pd.read_csv('/kaggle/input/mlb-fan-content-interaction/2017-mlb-homeruns.csv')
df3 = pd.read_csv('/kaggle/input/mlb-fan-content-interaction/2024-mlb-homeruns.csv')
df = pd.concat([df1, df2, df3], ignore_index=True)

# Prepare training data
batch_size = 50
num_batches = len(df) // batch_size + (len(df) % batch_size != 0)

# Create temp directory for videos
os.makedirs("temp_videos", exist_ok=True)

for batch_idx in range(num_batches):
    print(f"Processing batch {batch_idx + 1}/{num_batches}")
    df_batch = df[batch_idx * batch_size:(batch_idx + 1) * batch_size]
    X, y = [], []
    
    # Process videos with proper tqdm progress tracking
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(preprocess_video, row['video'], 'temp_videos/' + row['play_id']): row for _, row in df_batch.iterrows()}
        
        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing Videos"):
            res = future.result()
            if res is not None:
                X.append(res)
                y.append([futures[future]['ExitVelocity'], futures[future]['HitDistance'], futures[future]['LaunchAngle']])
    
    if len(X) == 0:
        print("Skipping batch due to no valid videos.")
        continue

    X = np.array(X)  # Shape: (batch_size, 10, 224, 224, 3)
    y = np.array(y)  # Shape: (batch_size, 3)

    print("Split data into train and test")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    print("Train model")
    model.fit(X_train, y_train, epochs=10, batch_size=8, validation_data=(X_test, y_test))

    print("Deleting Previous Saved model")
    if os.path.exists(f'model_batch_{batch_idx}.h5'):
        os.remove(f'model_batch_{batch_idx}.h5')
    
    print("Save model after each batch")
    model.save(f'model_batch_{batch_idx + 1}.h5')
    print(f"Model saved after batch {batch_idx + 1}")

    print("Evaluate model")
    loss, mae = model.evaluate(X_test, y_test)
    print(f'Batch {batch_idx + 1} - Test Loss: {loss}, Test MAE: {mae}')

    # Clear the TensorFlow session to release memory
    K.clear_session()
    
    # Trigger garbage collection to free up unused memory
    gc.collect()

    # Clear variables (X and y data) that are no longer needed
    X = []
    y = []


## Making Prediction by Loading Model

In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import os
import requests

# Load the trained model
model_path = "/kaggle/working/model_batch_8.h5"  # Replace 'X' with the latest batch number
model = tf.keras.models.load_model(model_path, compile=False)

# Recompile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='mse',  # Explicitly set the loss function again
              metrics=['mae'])

# Function to preprocess new video
def preprocess_video(video_url, save_path, frame_count=50):
    response = requests.get(video_url, stream=True)
    with open(save_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=1024):
            file.write(chunk)

    cap = cv2.VideoCapture(save_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_ids = np.linspace(0, total_frames - 1, frame_count, dtype=int)

    for fid in frame_ids:
        cap.set(cv2.CAP_PROP_POS_FRAMES, fid)
        ret, frame = cap.read()
        if not ret:
            continue
        frame = cv2.resize(frame, (224, 224)) / 255.0  # Normalize
        frames.append(frame)

    cap.release()
    os.remove(save_path)  # Remove temporary video file

    if len(frames) == frame_count:
        return np.array(frames)
    else:
        return None  # Return None if video does not have enough frames

# Path to the new video for prediction
row_index = 700
video_url = mlb_homeruns_df['video'][row_index]  # Replace with actual video URL
video_save_path = "temp_video.mp4"

# Preprocess video
video_frames = preprocess_video(video_url, video_save_path)

if video_frames is not None:
    video_frames = np.expand_dims(video_frames, axis=0)  # Add batch dimension (1, 50, 224, 224, 3)

    # Make prediction
    predictions = model.predict(video_frames)

    # Display predicted values
    print(f"Predicted ExitVelocity: {predictions[0][0]:.2f}")
    print(f"Predicted HitDistance: {predictions[0][1]:.2f}")
    print(f"Predicted LaunchAngle: {predictions[0][2]:.2f}")

    print(f"\n\nActual ExitVelocity: {mlb_homeruns_df['ExitVelocity'][row_index]}")
    print(f"Actual HitDistance: {mlb_homeruns_df['HitDistance'][row_index]}")
    print(f"Actual LaunchAngle: {mlb_homeruns_df['LaunchAngle'][row_index]}")
else:
    print("Error: Not enough valid frames in the video.")


In [ ]:
mlb_homeruns_df.head()

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os
import requests
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout, LSTM, TimeDistributed
from tensorflow.keras.optimizers import Adam
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import tensorflow.keras.backend as K
import gc

# Load pre-trained CNN model (feature extractor)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze pre-trained layers
feature_extractor = Model(inputs=base_model.input, outputs=Flatten()(base_model.output))  # Extracted features

# Define the LSTM-based regression model
model = Sequential([
    TimeDistributed(feature_extractor, input_shape=(60, 224, 224, 3)),  # Apply CNN to each frame
    LSTM(256, return_sequences=False),  # LSTM processes extracted features
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(3)  # Predict ExitVelocity, HitDistance, LaunchAngle
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
model.summary()

# Function to download video
def download_video(video_url, save_path):
    response = requests.get(video_url, stream=True)
    with open(save_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=1024):
            file.write(chunk)
    return save_path

# Function to extract frames from video
def preprocess_video(video_url, save_path, frame_count=60):
    video_path = download_video(video_url, f'{save_path}.mp4')
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_ids = np.linspace(0, total_frames - 1, frame_count, dtype=int)  # Sample evenly
    
    for fid in frame_ids:
        cap.set(cv2.CAP_PROP_POS_FRAMES, fid)
        ret, frame = cap.read()
        if not ret:
            continue
        frame = cv2.resize(frame, (224, 224)) / 255.0  # Normalize
        frames.append(frame)
    
    cap.release()
    os.remove(video_path)  # Remove temp file
    return np.array(frames) if len(frames) == frame_count else None 

# Load and merge datasets
df1 = pd.read_csv('/kaggle/input/mlb-fan-content-interaction/2016-mlb-homeruns.csv')
df2 = pd.read_csv('/kaggle/input/mlb-fan-content-interaction/2017-mlb-homeruns.csv')
df3 = pd.read_csv('/kaggle/input/mlb-fan-content-interaction/2024-mlb-homeruns.csv')
df = pd.concat([df1, df2, df3], ignore_index=True)

# Prepare training data
batch_size = 30
num_batches = len(df) // batch_size + (len(df) % batch_size != 0)

# Create temp directory for videos
os.makedirs("temp_videos", exist_ok=True)

for batch_idx in range(num_batches):
    print(f"Processing batch {batch_idx + 1}/{num_batches}")
    df_batch = df[batch_idx * batch_size:(batch_idx + 1) * batch_size]
    X, y = [], []
    
    # Process videos with proper tqdm progress tracking
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(preprocess_video, row['video'], 'temp_videos/' + row['play_id']): row for _, row in df_batch.iterrows()}
        
        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing Videos"):
            res = future.result()
            if res is not None:
                X.append(res)
                y.append([futures[future]['ExitVelocity'], futures[future]['HitDistance'], futures[future]['LaunchAngle']])
    
    if len(X) == 0:
        print("Skipping batch due to no valid videos.")
        continue

    X = np.array(X)  # Shape: (batch_size, 10, 224, 224, 3)
    y = np.array(y)  # Shape: (batch_size, 3)

    print("Split data into train and test")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    print("Train model")
    model.fit(X_train, y_train, epochs=10, batch_size=8, validation_data=(X_test, y_test))

    print("Deleting Previous Saved model")
    if os.path.exists(f'model_batch_{batch_idx}.h5'):
        os.remove(f'model_batch_{batch_idx}.h5')
    
    print("Save model after each batch")
    model.save(f'model_batch_{batch_idx + 1}.h5')
    print(f"Model saved after batch {batch_idx + 1}")

    print("Evaluate model")
    loss, mae = model.evaluate(X_test, y_test)
    print(f'Batch {batch_idx + 1} - Test Loss: {loss}, Test MAE: {mae}')

    # Clear the TensorFlow session to release memory
    K.clear_session()
    
    # Trigger garbage collection to free up unused memory
    gc.collect()

    # Clear variables (X and y data) that are no longer needed
    X = []
    y = []


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ time_distributed (TimeDistributed)   │ (None, 60, 100352)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 256)                 │     103,023,616 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │           1,539 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 126,744,451 (483.49 MB)

...

Processing Videos: 100%|██████████| 30/30 [03:15<00:00,  6.51s/it]


Split data into train and test
Train model
Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 147s 34s/step - loss: 574.7592 - mae: 14.6953 - val_loss: 185.3759 - val_mae: 9.4408
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 4s/step - loss: 581.2664 - mae: 14.6124 - val_loss: 185.9819 - val_mae: 9.4040
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 4s/step - loss: 485.8528 - mae: 13.4840 - val_loss: 187.5406 - val_mae: 9.4066
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 4s/step - loss: 470.8186 - mae: 13.3196 - val_loss: 188.6086 - val_mae: 9.5420
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 4s/step - loss: 562.8091 - mae: 15.4169 - val_loss: 187.6433 - val_mae: 9.4506
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 4s/step - loss: 519.8569 - mae: 15.3843 - val_loss: 185.6766 - val_mae: 9.4699
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 4s/step - loss: 527.9682 - mae: 15.0887 - val_loss: 184.7634 - val_mae: 9.6079
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 4s/step - loss: 392.7201 - mae: 12.3091 - val_loss: 184.4211 - val_mae: 9.6665
Epo

Processing Videos: 100%|██████████| 30/30 [03:17<00:00,  6.58s/it]


Split data into train and test
Train model
Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 146s 34s/step - loss: 387.3683 - mae: 12.5657 - val_loss: 121.2895 - val_mae: 7.3121
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 4s/step - loss: 494.1400 - mae: 14.3322 - val_loss: 125.3732 - val_mae: 7.3387
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 4s/step - loss: 385.5602 - mae: 13.9369 - val_loss: 133.8627 - val_mae: 7.6079
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 4s/step - loss: 323.8651 - mae: 12.1799 - val_loss: 139.1204 - val_mae: 7.7830
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 4s/step - loss: 323.9953 - mae: 11.1509 - val_loss: 142.7835 - val_mae: 7.8872
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 4s/step - loss: 380.9462 - mae: 12.6862 - val_loss: 145.2580 - val_mae: 7.9860
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 4s/step - loss: 378.1563 - mae: 13.4197 - val_loss: 141.3626 - val_mae: 7.8041
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 4s/step - loss: 483.5509 - mae: 14.8976 - val_loss: 137.1983 - val_mae: 7.6491
Epo

Processing Videos:  67%|██████▋   | 20/30 [02:13<00:14,  1.41s/it]